<a href="https://colab.research.google.com/github/young-hwanlee/stand_alone_deep_learning/blob/main/%2312_Run_PyTorch_on_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# !pip install -q torch==1.0.0 torchvision
import torch
print(torch.__version__)

1.12.0+cu113


In [11]:
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np

In [12]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_set, val_set = torch.utils.data.random_split(train_set, [40000, 10000])
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=4,
                                           shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=4,
                                         shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4,
                                          shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [13]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, n_layer, act):
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layer = n_layer
        self.act = act

        self.fc = nn.Linear(self.input_dim, self.hidden_dim)
        self.linears = nn.ModuleList()
        for i in range(self.n_layer - 1):
            self.linears.append(nn.Linear(self.hidden_dim, self.hidden_dim))
            
        self.fc2 = nn.Linear(self.hidden_dim, self.output_dim)

        if self.act == 'relu':
            self.act = nn.ReLU()
    
    def forward(self, x):
        x = self.act(self.fc(x))
        for fc in self.linears:
            x = self.act(fc(x))
        x = self.fc2(x)
        return x

In [14]:
net = MLP(input_dim=3072, output_dim=10, hidden_dim=100, n_layer=4, act='relu')

In [15]:
def experiment(args):

    net = MLP(args.input_dim, args.output_dim, args.hidden_dim, args.n_layer, args.act)
    net.cuda()
    print(net)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(args.epoch):

        # ===== Train ===== #
        net.train()

        running_loss = 0.0
        train_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            optimizer.zero_grad()

            # get the inputs
            input, label = data
            input = input.view(-1, 3072)

            input = input.cuda()
            label = label.cuda()

            # forward + backward + optimize
            output = net(input)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            train_loss += loss.item()

            # print the results every 2000 mini-batches
            if i % 2000 == 1999:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
        
        # ===== Validate ===== #
        net.eval()

        correct = 0
        total = 0
        val_loss = 0
        with torch.no_grad():
            for data in val_loader:
                image, label = data
                image = image.view(-1, 3072)

                image = image.cuda()
                label = label.cuda()

                output = net(image)

                loss = criterion(output, label)
                val_loss += loss.item()

                _, predicted = torch.max(output.data, 1)
                total += label.size(0)
                correct += (predicted == label).sum().item()
            
            val_loss = val_loss / len(val_loader)
            val_acc = 100 * correct / total
        
        print('Epoch: {}, Train Loss: {}, Validation Loss: {}, Validation Accuracy: {}'.format(epoch + 1, train_loss, val_loss, val_acc))
    
    # ===== Test ===== #
    net.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            image, label = data
            image = image.view(-1, 3072)
            
            image = image.cuda()
            label = label.cuda()

            output = net(image)
            _, predicted = torch.max(output.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
        
        test_acc = 100 * correct / total
    
    return train_loss, val_loss, val_acc, test_acc

In [16]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.n_layer = 5
args.input_dim = 3072
args.output_dim = 10
args.hidden_dim = 100
args.act = 'relu'

args.lr = 0.001
args.momentum = 0.9
args.epoch = 2

list_var1 = [4, 5, 6]
list_var2 = [50, 100, 150]

for var1 in list_var1:
    for var2 in list_var2:
        args.n_layer = var1
        args.hidden_dim = var2
        result = experiment(args)
        print(result)

MLP(
  (fc): Linear(in_features=3072, out_features=50, bias=True)
  (linears): ModuleList(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
  )
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (act): ReLU()
)
[1,  2000] loss: 2.194
[1,  4000] loss: 1.902
[1,  6000] loss: 1.759
[1,  8000] loss: 1.705
[1, 10000] loss: 1.675
Epoch: 1, Train Loss: 18469.89679786563, Validation Loss: 1.6800018454790115, Validation Accuracy: 39.51
[2,  2000] loss: 1.620
[2,  4000] loss: 1.618
[2,  6000] loss: 1.583
[2,  8000] loss: 1.559
[2, 10000] loss: 1.541
Epoch: 2, Train Loss: 15842.223545625806, Validation Loss: 1.5573715364336969, Validation Accuracy: 44.1
(15842.223545625806, 1.5573715364336969, 44.1, 45.04)
MLP(
  (fc): Linear(in_features=3072, out_features=100, bias=True)
  (linears): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)